## Map/Reduce Semantics and Systems

### Types and transformations
* Map is a transformation
  * Input domain to output domain
* Reduce is a collection
  * No domain change
  
$$
map (k1, v1) \rightarrow list(k2,v2) \\
reduce (k2, list(v2)) \rightarrow list(v2)
$$

* Google C++ implementation is based all on strings
  * User code must convert to structured types
* Hadoop! has type wrappers

### Parallelism in Map/Reduce

* How much potential paralleism in mappers? in reducers?


.

.

......spoiler alert.......
  
.  

.
* Mappers: up to a parallel process for each input (typically a file)
* Reducers: up to a parallel process for each key
* So for the WordCount example
  * two files = two mappers
  * 5 different words = five reducers
  * but this is scalable with input


Differentiating betweens mapper/reducer and map/reduce processes
* A cluster will typically configure number of available phyical processes
  * this number is typically much smaller than potential parallelism
  * we refer to `mappers` as potential parallelism and `map processes` as the number of phyical processes running map functions.

### Map/Reduce Runtimes

From the Google paper https://www.usenix.org/legacy/events/osdi04/tech/dean.html

<img src="images/mr.png" width=512 />

* Automatically partition input data
  * 16-64 MB chunks for Google
* Create M map tasks: one for each chunk
  * Assign available workers (up to M) to tasks
* Write intermediate pairs to local (to worker) disk
* R reduce tasks (defined by user) read and process intermediate results
* Output is up to R files available on shared file system
* Master tracks state
  * Asssignment of M map tasks and R reduce tasks to workers
  * State and liveliness of the above


### Systems Issues

The map/reduce runtime must deal with:
* Master failure
  * Checkpoint/restart, classic distributed systems/replication problem
* Failed worker
  * Heartbeat liveness detection, restart
* Slow worker
  * Backup tasks
* Locality of processing to data
  * Big deal, they don’t really solve
  * But, much subsequent research does
* Task granularity
  * Metadata size and protocol scaling (not inherent parallelism) limit the size of M and R
  
### Google File System: The Data Service

* Goals
  * Wide-distribution
  * Commodity hardware
  * High (aggregate) performance
* Different assumptions than traditional file systems
  * Component failures are normal behavior
  * Files are huge (new to Google environment ca. 2004)
* Most files have append-only writing, 
  * Mandate append-only writing to realize good I/O properties
  * Why append only?
      * reduce contention -- logical locking of tail rather than physcial locking of offset
      * no data reorganization for writes

GFS architecture (from https://www.cs.rutgers.edu/~pxk/417/notes/16-dfs.html)
    
<img src="images/gfs.png" width=512 />

<img src="images/gfs2.png" width=512 />


*  Reliable services
  * Master, scheduler, lock services fault tolerant
* Data are triple replicated
  * On nodes that have independent failure properties (different racks, power supplies, networks)
  * This became standard practice in cloud key/value stores (for a decade, now supersedes by distributed error coding)
  * Tolerates two failures
  * Rereplicated on failure detection
  
<!--
Here is the originial diagram from https://www.usenix.org/legacy/events/osdi04/tech/dean.html

<img src="images/gfs.orig.png" width=700 />
-->

### How GFS changed the world

* Atomic checkpoint and append
  * Major mode for writing
  * Great semantics for limited usage
* Abandon POSIX file system semantics
* In-memory metadata at Master
  * Gotta keep it small
  * Even for big data (scale metadata memory in proportion to aggregate storage)
* Re-replication
  * Keep three, detect missing on read/write
  * Forget reliable storage, forget RAID
* Design for failures (not recovery)